# Memory and resource deallocation test.

This notebook uses the NER example code to test for resource leaks.  Please see the `ner.ipynb` for the actual NER notebook example.

In [ ]:
# set up notebook environment
import sys
from pathlib import Path
app_root_dir = Path('..')
# add the example to the Python library path
sys.path.append(str(app_root_dir / 'src'))
# add the deepnlp path
sys.path.append('../../../src/python')

## Configure and create the app specific facade

Now that the interpreter environment is set up, we can import local packages.

In [ ]:
from zensols.deeplearn import TorchConfig
from zensols.deeplearn.cli import JupyterManager
from movie import CliFactory

# reset random state for consistency of each new test that uses this function
TorchConfig.init()

mng = JupyterManager(
    allocation_tracking=True,
    cli_class=CliFactory,
    factory_args={'root_dir': app_root_dir},
    cli_args_fn=lambda model: ['-c', str(app_root_dir / 'models' / f'{model}.conf')])
    
# number of epochs to test with, which needs to be at least two to excercise it properly
n_epochs = 2
# if a memory leak test fails, set to False for that test
keep_going = True

In [ ]:
from io import StringIO
facade = mng.create_facade('glove50')
facade.epochs = n_epochs
# write to force more allocation to test deallocation, but don't clutter the notebook output
facade.write(writer=StringIO())
mng.run(False)

In [ ]:
if keep_going:
    facade.embedding = 'glove_300_embedding'
    mng.run(False)
    mng.show_leaks()

In [ ]:
if keep_going:
    facade = mng.create_facade('transformer-fixed')
    facade.epochs = n_epochs
    mng.run(False)

In [ ]:
if keep_going:
    facade = mng.create_facade('transformer-trainable')
    facade.epochs = n_epochs
    mng.run(False)
    mng.show_leaks()